# Exploring the gliders ERDDAP server

In [ ]:
from erddapy import ERDDAP


e = ERDDAP(
    server='https://data.ioos.us/gliders/erddap',
    protocol='tabledap',
)

### Define filter criteria:

Using the erddapy library to query the IOOS Glider DAC ERDDAP server, we'll look for CF standard name 'sea_water_temperature' with a bounding box covering roughly the Gulf of Mexico year 2018.  Also, filter by CDM data type of 'trajectoryprofile'.


In [ ]:
min_lat, max_lat = 20, 30
min_lon, max_lon = -82, -97
        
kw = {
    'standard_name': 'sea_water_temperature',
    'min_lon': min_lon,
    'max_lon': max_lon,
    'min_lat': min_lat,
    'max_lat': max_lat,
    'min_time': '2018-01-10T00:00:00Z',
    'max_time': '2018-09-01T00:00:00Z',
    'cdm_data_type': 'trajectoryprofile'
}

erddapy generates an ERDDAP 'advanced' search-compliant search url for us automatically (preview results using the link below):

In [ ]:
search_url = e.get_search_url(response='csv', **kw)

print('search_url: ' + search_url)

### Pandas integration:

Interrogate the search results using a Pandas DataFrame created from ERDDAP's csv output format:

In [ ]:
import pandas as pd

search = pd.read_csv(search_url)
gliders = search['Dataset ID'].values

print('Found {} Glider Datasets:\n{}'.format(len(gliders), '\n'.join(gliders)))

Pick a dataset and associated variables of interest:

In [ ]:
dataset_id = 'sam-20180824T0000'


variables = [
 'depth',
 'latitude',
 'longitude',
 'salinity',
 'temperature',
 'time',
]

Assign these values to the ERRDAP server instance to download the glider deployment data!

Filipe, it would be a great addition to the erddapy library if there were a way to break down the query/download flow somehow so you could first interrogate a dataset's metadata (just the e.dataset_id = dataset_id) part, output metadata showing available variables, then apply a variable filter on a dataset object, rather than all the way up at the 'e' server level.  Such as:

```
e.dataset_id = dataset_id


dataset = e.get_dataset()
dataset                   #show variables, global attributes/other metadata
```
```
dataset.variables = variables

data = d.get_download_url(
        response='mat',  # like matlab? ERDDAP has your back!
    )
```
etc..

This may not be possible wrt ERDDAP's capabilities though, IDK.  Conceptually, that would make this much clearer and more intuitive though.  

In [ ]:
e.dataset_id = dataset_id
e.variables = variables

print(
    e.get_download_url(
        response='mat',  # like matlab? ERDDAP has your back!
    )
)

Clicking in the URL above will get a `.mat` file ready for download.
However, we'll stay in the Python world and use pandas to load a `csv`.

In [ ]:
df = e.to_pandas(
    index_col='time',
    parse_dates=True,
    skiprows=(1,)
).dropna()

df.head()

### Plot a trajectoryProfile of water temperature using matplotlib



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

fig, ax = plt.subplots(figsize=(17, 2))
cs = ax.scatter(df.index, df['depth'], s=15, c=df['temperature'],
                marker='o', edgecolor='none')

ax.invert_yaxis()
ax.set_xlim(df.index[0], df.index[-1])
xfmt = mdates.DateFormatter('%H:%Mh\n%d-%b')
ax.xaxis.set_major_formatter(xfmt)

cbar = fig.colorbar(cs, orientation='vertical', extend='both')
cbar.ax.set_ylabel('Temperature ($^\circ$C)')
ax.set_ylabel('Depth (m)');

### Plot the glider track using CartoPY:

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature


land_10m = cfeature.NaturalEarthFeature(
    'physical', 'land', '10m',
     edgecolor='k', facecolor='0.75',
)


fig, ax = plt.subplots(
    subplot_kw={'projection': ccrs.PlateCarree()},
    figsize=(7, 7)
)

ax.plot(df['longitude'], df['latitude'], 'o')

gl = ax.gridlines(draw_labels=True)
gl.xlabels_bottom = gl.ylabels_left = False

ax.add_feature(land_10m)

dx = dy = 2
ax.set_extent(
    [df['longitude'].min()-dx, df['longitude'].max()+dx,
     df['latitude'].min()-dy, df['latitude'].max()+dy]
);